In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
### load the trained mode, scaler, pickle, one  hot
model=load_model('model.h5')

## load the encoders and scalar
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scalar.pkl','rb') as file:
    scalar=pickle.load(file)

In [4]:
## Example input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':60,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [ ]:
# One hot encode Geography
input_data=pd.DataFrame([input_data])
geo_encoded = onehot_encoder_geo.transform(input_data[['Geography']])

geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)

geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [16]:
input_df=pd.DataFrame(input_data)
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,60,3,60000,2,1,1,50000


In [17]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,60,3,60000,2,1,1,50000


In [18]:
## concatenation onehot encoded data
input_df=pd.concat([input_df.drop("Geography",axis=1), geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,60,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
## Scaling the input data
input_scaled=scalar.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  2.00624232, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [21]:
# Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step


array([[0.10442554]], dtype=float32)

In [22]:
prediction_proba=prediction[0][0]
prediction_proba

0.10442554

In [23]:
if prediction_proba>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
